<a href="https://colab.research.google.com/github/sagitiminsky/Final-Project-in-Advanced-Lectures-in-Learning-Theory/blob/main/part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("this is a test")

this is a test
